# NOTE:
The key different between agent as tool (.as_tool method use in the previous lab) is that:
  - the execution flow is hand from one agent to another, the other agent take over the workflow
  - whereas agent as tool call another agent, and continue the flow

In [ ]:
# LESSON:
hand off require explicit what data to be sent, here is except from the answer sheet:
```
Pass ONLY the winning email draft to the 'Email Manager' agent
```
and
```
must hand off exactly ONE email to the Email Manager — never more than one.
```

Without these explicit instruction, the email agent may be hand offed without any data to run.



In [1]:
from dotenv import load_dotenv

from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent

import os
import asyncio
from typing import Dict


load_dotenv(override=True)
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from  trd_agent import ChatOllama, ChatDeepSeek

llm_deepseek = ChatDeepSeek(model="deepseek-chat", model_tag="deepseek", temperature=0)

In [3]:
instruction1 = "you are a sales agent working for ComplAI, \
a company providing SaaS for ensuring SOC2 compliance, and preparing for audit \
power y AI. You write professional, serios, cold email"

instruction2 = "you are a humorous, engaging sales agent working for ComlAI, \
a company providing SaaS for ensuring SOC2 compliance, and preparing for audit \
You write witty, engaging cold emails that are likely to get a response"

instruction3 = "you are a busy sales agent working for ComlAI, \
a company providing SaaS for ensuring SOC2 compliance, and preparing for audit \
You write concise, to the point cold email."

sales_agent1 = Agent(name="Professional Sales Agent", instructions=instruction1, model="gpt-4o-mini")
sales_agent2 = Agent(name="Professional Sales Agent", instructions=instruction2, model="gpt-4o-mini")
sales_agent3 = Agent(name="Professional Sales Agent", instructions=instruction3, model="gpt-4o-mini")

tool_sale1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool_sale2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description="Write a cold sales email")
tool_sale3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description="Write a cold sales email")

@function_tool
def send_email(body: str):
    print ("pretending to send an email")
    print ("sending....")
    print (f"\n\n#############{body}###########\n\n")
    print ("done!")
    return {"status": "success"}

writing_tools = [tool_sale1, tool_sale2, tool_sale3]

writing_instruction = """You are a sale manager working for ComlAI. Use the tools given to generate cold sale emails.
Never generate sale emails by yourself; alway use tools.
Try all 3 sale agent tools, each tool only once.
Pick a single best email from your own judment of which email will be most effective.
After picking the best email, hand off to the Email Manager."""

writing_instruction = """You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


In [4]:
subject_instruction = "Given a message, you write a subject for a cold sales email that is likely to get a response"

html_instruction = "Given a text email body which may have some markdown, convert it to an HTML email with simple, clear, compeling layout and design"


subject_writer = Agent(name="Email subject writer", instructions=subject_instruction, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name = "subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instruction, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter", tool_description="Convert a text email body to an HTML email body")

@function_tool
def send_html_email(subject: str, html_body: str)->Dict[str,str]:
    """ Send out an email with given subject and HTML body to all sales prospect """
    print ("pretending to send an html email")
    print ("sending....")
    print (f"\n\n#############{subject}###########\n\n")
    print (f"\n\n#############{html_body}###########\n\n")
    print ("done!")
    return {"status": "success"}

email_tools = [subject_tool, html_tool, send_html_email]

email_tools


[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x703d11dc71c0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x703d11dc7520>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 Fu

In [5]:
email_instruction="""You are email formatter and sender.
Given body of an email to be sent, you first use the subject_writer tool to write subject for the email.
Then, use the html_converter tool to convert to body to HTML.
Finally, use send_html_email tool to send out the email with subject and HTML body."""
emailer_agent = Agent(name="Email Manager", instructions=email_instruction, tools=email_tools, model="gpt-4o-mini", handoff_description="Convert an email to HTML and send it")

handoff = [emailer_agent]

print(handoff)

[Agent(name='Email Manager', handoff_description='Convert an email to HTML and send it', tools=[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x703d11dc71c0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x703d11dc7520>, strict_js

In [6]:

sales_manager = Agent(name="Sale Manager", instructions=writing_instruction, model="gpt-4o-mini", tools=writing_tools, handoffs=handoff)
message = "Send a cold sale email addressed to 'Dear CEO'"

In [7]:
print (sales_manager)

Agent(name='Sale Manager', handoff_description=None, tools=[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x703d11dc64d0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x703d11dc6830>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrai

In [8]:
with trace("Automated SDR handoff"):
    result = await Runner.run(sales_manager, message)

pretending to send an html email
sending....


#############Elevate Your SOC 2 Compliance Efforts with ComplAI###########




#############<p>Dear CEO,</p>
<p>I hope this message finds you well.</p>
<p>In today’s competitive landscape, ensuring robust compliance with industry standards like SOC 2 is not just a regulatory checkbox; it’s a strategic imperative that can significantly enhance your organization’s credibility and trustworthiness.</p>
<p>At ComplAI, we specialize in providing a powerful SaaS solution that streamlines the SOC 2 compliance process. Our AI-driven platform not only simplifies documentation and auditing but also significantly reduces the time and resources typically required for compliance preparation. With ComplAI, your team can focus on core business activities, while we handle the complexities of compliance management.</p>
<p><strong>Key benefits of our solution include:</strong></p>
<ul>
<li><strong>Automated Documentation:</strong> Generate and maintain compl

In [9]:
print (result.final_output)

The email has been successfully sent with the following details:

**Subject:** Elevate Your SOC 2 Compliance Efforts with ComplAI

**HTML Body:**
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Email</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f4f4f4;
        }
        .container {
            max-width: 600px;
            margin: auto;
            background: white;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);
        }
        h2 {
            color: #333;
        }
        p {
            line-height: 1.6;
            color: #555;
        }
        ul {
            list-style-type: none;
            padding: 0;
        }
        li {
            margin: 10px 0;
        }
        strong {
     

In [10]:
#!jupyter nbconvert --to python lab3_agent_handoff.ipynb